# 계산된 tf_idf 파일에 대한 거래량과 상관관계 분석

point biserial correlation

In [91]:
from scipy.stats import pointbiserialr

In [94]:
import numpy as np

In [1]:
import pandas as pd

tf_idf 결과값 데이터 로드

In [40]:
data = pd.read_csv("./user_item_via_tf_idf_ver2.csv", encoding='utf-8', index_col=0)

In [41]:
data.shape

(38564, 4040)

In [42]:
data.head()

,1_B01,2_B01,3_B01,4_A01,5_B01,5_B02,6_B03,6_A01,6_B02,6_A03,...,1664_A01,1664_A02,1665_B01,1665_A01,1666_A02,1666_A01,1666_B01,1667_A01,1667_B01,1667_A02
clnt_id,,,,,,,,,,,,,,,,,,,,,
1,0.057889,0.000000,0.000000,0.000000,0.307914,0.0,0.000000,0.000000,0.000000,0.000000,...,0.059934,0.563261,0.0,0.015307,1.170073,0.492548,0.542039,0.0,0.357980,0.355827
2,0.167871,0.040374,0.192273,0.000000,0.614879,0.0,0.056325,0.036712,0.031314,0.018003,...,0.061681,1.677228,0.0,0.000000,3.076033,0.404429,1.165988,0.0,1.581158,0.471470
3,0.103366,0.000000,0.000000,0.058457,0.183617,0.0,0.000000,0.000000,0.000000,0.000000,...,0.089014,1.074976,0.0,0.000000,1.139756,0.223298,0.265743,0.0,0.035172,0.336807
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,0.052923,0.000000,0.000000,0.000000,0.538846,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.423454,0.0,0.000000,1.392167,0.204447,0.342597,0.0,0.347927,0.161338


거래 정보 데이터 로드

In [7]:
deal_info = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-02.거래 정보.csv')

In [9]:
deal_info.head(10)

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
0,21922,104999,1,A03,unknown,20190920,12:41,5990,1
1,21279,104907,4,A03,unknown,20190920,10:27,10900,1
2,39423,105124,11,A03,unknown,20190920,17:26,12900,1
3,18362,104010,1,A03,unknown,20190920,09:57,9900,1
4,39423,105124,13,A03,0565,20190920,17:26,2990,1
5,21279,104907,1,A03,0565,20190920,10:27,2990,1
6,48969,104882,5,A03,0572,20190920,10:05,7490,1
7,30533,103245,5,A03,0670,20190920,14:36,49800,1
8,64346,104317,1,A03,0543,20190920,22:34,26900,1
9,58071,104894,2,A03,unknown,20190920,10:12,4480,1


In [16]:
'543_A03' in list(data.columns)[1:]

True

In [18]:
# unknown 제거
deal_info = deal_info[deal_info.pd_c != 'unknown']

In [20]:
# string->integer 변환
def remove_first_zero(row):
    a = row
    if row[0] == '0':
        a = row[1:]
    return a

In [21]:
deal_info['pd_c'] = deal_info['pd_c'].apply(remove_first_zero)

In [22]:
deal_info.head()

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
4,39423,105124,13,A03,565,20190920,17:26,2990,1
5,21279,104907,1,A03,565,20190920,10:27,2990,1
6,48969,104882,5,A03,572,20190920,10:05,7490,1
7,30533,103245,5,A03,670,20190920,14:36,49800,1
8,64346,104317,1,A03,543,20190920,22:34,26900,1


In [25]:
# 정렬
deal_info = deal_info.sort_values(by=['clnt_id'])

In [26]:
deal_info.head()

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct
118651,2,64691,4,A03,189,20190731,21:25,4290,1
118652,2,64691,1,A03,186,20190731,21:25,3790,1
107209,2,64691,2,A03,151,20190731,21:25,3990,1
114604,2,62037,1,A03,092,20190729,23:47,36000,20
161808,2,88495,1,A03,1395,20190830,22:00,3590,1


In [57]:
# data와 칼럼 값 맞추기
deal_info['item'] = deal_info['pd_c']+'_'+deal_info['biz_unit']

In [58]:
deal_info.head()

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct,item
114604,2,62037,1,A03,92,20190729,23:47,36000,20,92_A03
583390,2,42449,1,A02,1015,20190704,15:34,46430,1,1015_A02
583389,2,72330,1,A02,857,20190809,10:14,34970,1,857_A02
113315,2,64691,3,A03,351,20190731,21:25,4690,1,351_A03
161808,2,88495,1,A03,1395,20190830,22:00,3590,1,1395_A03


In [61]:
dic = dict()

for i in list(data.index):
    tmp = dict()
    for goods in list(data.columns):
        tmp[goods]=0
    dic[i] = tmp

In [62]:
# 구매 건수 count한 딕셔너리 생성

for idx, row in deal_info.iterrows():
    try:
        dic[row['clnt_id']][row['item']] += 1
    except:
        pass

In [99]:
dic[2]

{'1_B01': 0,
 '2_B01': 0,
 '3_B01': 0,
 '4_A01': 0,
 '5_B01': 0,
 '5_B02': 0,
 '6_B03': 0,
 '6_A01': 0,
 '6_B02': 0,
 '6_A03': 0,
 '6_B01': 0,
 '7_A01': 0,
 '7_B01': 0,
 '8_B01': 0,
 '9_B01': 0,
 '10_B01': 0,
 '11_B01': 0,
 '12_B01': 0,
 '13_B01': 0,
 '13_A01': 0,
 '14_B01': 0,
 '15_B01': 0,
 '16_A02': 0,
 '17_A02': 0,
 '17_A01': 0,
 '18_B01': 0,
 '19_B01': 0,
 '20_B01': 0,
 '21_B01': 0,
 '22_B01': 0,
 '23_B02': 0,
 '23_B01': 0,
 '23_A03': 0,
 '24_A01': 0,
 '24_B01': 0,
 '25_B01': 0,
 '26_B01': 0,
 '27_A03': 0,
 '27_B01': 0,
 '27_B02': 0,
 '28_B01': 0,
 '29_B01': 0,
 '30_B02': 0,
 '30_A03': 0,
 '30_B01': 0,
 '31_A01': 0,
 '31_B01': 0,
 '31_A03': 0,
 '31_A02': 0,
 '31_B02': 0,
 '32_A03': 0,
 '32_B01': 0,
 '32_B02': 0,
 '32_A02': 0,
 '33_A03': 0,
 '33_B02': 0,
 '33_B01': 0,
 '34_B01': 0,
 '35_B01': 0,
 '35_A01': 0,
 '36_A01': 0,
 '36_B01': 0,
 '37_B01': 0,
 '38_B01': 0,
 '39_B01': 0,
 '40_B01': 0,
 '41_B01': 0,
 '42_B01': 0,
 '43_A01': 0,
 '43_A02': 0,
 '44_A01': 0,
 '44_A02': 0,
 '44_B0

In [100]:
data.columns

Index(['1_B01', '2_B01', '3_B01', '4_A01', '5_B01', '5_B02', '6_B03', '6_A01',
       '6_B02', '6_A03',
       ...
       '1664_A01', '1664_A02', '1665_B01', '1665_A01', '1666_A02', '1666_A01',
       '1666_B01', '1667_A01', '1667_B01', '1667_A02'],
      dtype='object', length=4040)

In [107]:
# 상관계수와 p-value를 저장할 리스트 생성
cor = []
pval = []

In [108]:
for clnt_id in key_lst:
    print(clnt_id)
    tf_idf = np.array(list(data.iloc[clnt_id])).astype(np.float)
    real_buy = np.array(list(dic[clnt_id].values())).astype(np.float)
    pbc = pointbiserialr(tf_idf,real_buy)
    cor.append(pbc[0])
    pval.append(pbc[1])    

1
2
3
4
6
7
8
9
10
11
12
13
14
16
17
18
21
22
23
24
25
26
27
28
29
30
31
33
34
35
36
37
38
40
41
42
43
44
48
49
50
52
53
55
56
61
65
68
69
72
73
75
80
84
85
88
89
90
95
99
101
104
105
106
107
116
121
123
124
128
129
133
134
136
137
141
142
145
148
149
150
151
152
153
154
155
156
157
159
162
164
167
169
172
173
174
175
176
177
179
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
197
198
199
200
201
202
204
205
206
207
208
209
210
211
212
213
214
215
217
218
219
222
223
225
227
228
229
230
232
233
235
236
237
238
239
240
241
242
243
244
246
247
248
249
250
251
253
258
263
268
269
270
271
274
275
277
279
282
287
288
290
294
298
305
307
312
315
316
319
320
323
324
325
332
333
336
338
343
344
346
348
349
350
353
357
358
361
362
363
364
368
371
373
376
377
378
379
383
386
387
388
390
391
392
393
394
395
396
397
399
404
407
408
409
412
414
415
416
417
419
420
422
423
426
427
431
432
434
435
437
440
442
445
447
448
451
452
453
455
456
457
458
459
460
462
464
465
466
468
469
470
473


5423
5424
5425
5427
5428
5430
5431
5432
5434
5435
5436
5438
5440
5441
5443
5444
5446
5449
5450
5451
5452
5453
5459
5461
5462
5466
5469
5470
5471
5472
5474
5476
5481
5482
5486
5488
5490
5491
5493
5494
5495
5496
5497
5498
5499
5500
5502
5503
5504
5505
5506
5507
5508
5510
5511
5512
5513
5514
5515
5516
5517
5518
5519
5520
5522
5523
5524
5525
5526
5527
5528
5529
5531
5532
5533
5534
5535
5536
5537
5538
5539
5540
5541
5542
5543
5544
5545
5547
5548
5553
5557
5558
5559
5560
5561
5562
5563
5565
5567
5570
5572
5573
5574
5576
5579
5581
5582
5583
5584
5585
5588
5590
5591
5592
5593
5595
5598
5599
5600
5602
5606
5609
5612
5614
5615
5617
5619
5620
5623
5624
5626
5630
5631
5634
5635
5637
5638
5641
5643
5645
5646
5656
5658
5659
5666
5667
5670
5675
5678
5682
5685
5687
5689
5690
5693
5695
5696
5697
5701
5703
5708
5709
5710
5711
5712
5715
5717
5719
5721
5722
5723
5724
5726
5727
5728
5732
5733
5734
5735
5737
5739
5740
5741
5742
5743
5744
5745
5746
5747
5748
5749
5750
5751
5752
5753
5754
5756
5757
5760
5761


10561
10562
10563
10565
10567
10568
10569
10570
10571
10572
10573
10575
10579
10580
10581
10582
10584
10585
10586
10587
10588
10590
10591
10594
10595
10596
10597
10599
10600
10604
10605
10606
10607
10608
10609
10610
10611
10612
10613
10614
10615
10616
10617
10618
10623
10626
10636
10639
10640
10641
10642
10648
10653
10656
10658
10662
10663
10668
10670
10675
10677
10678
10679
10680
10682
10684
10685
10689
10692
10697
10699
10700
10701
10702
10704
10705
10706
10707
10708
10710
10711
10712
10713
10714
10718
10720
10722
10725
10731
10733
10741
10742
10743
10746
10748
10749
10752
10754
10759
10762
10766
10774
10776
10777
10780
10785
10786
10790
10791
10792
10795
10797
10798
10800
10802
10803
10804
10805
10808
10809
10811
10812
10815
10821
10822
10823
10824
10826
10829
10830
10833
10836
10841
10842
10843
10845
10846
10851
10852
10855
10856
10860
10870
10871
10879
10880
10882
10883
10887
10889
10890
10891
10894
10895
10896
10898
10899
10905
10907
10913
10916
10917
10919
10920
10925
10926
1092

15861
15862
15864
15865
15869
15875
15876
15879
15882
15884
15885
15888
15891
15895
15897
15900
15901
15904
15905
15906
15910
15912
15920
15921
15923
15927
15930
15931
15939
15940
15942
15945
15954
15956
15960
15961
15964
15965
15970
15972
15975
15976
15977
15978
15979
15980
15981
15982
15983
15984
15985
15986
15987
15988
15989
15990
15991
15992
15993
15994
15997
15998
15999
16001
16005
16006
16007
16009
16010
16011
16012
16018
16021
16028
16031
16038
16039
16040
16041
16043
16044
16046
16047
16048
16050
16052
16053
16054
16056
16060
16065
16069
16075
16076
16077
16079
16083
16084
16091
16092
16095
16098
16102
16104
16108
16111
16113
16117
16120
16123
16124
16125
16126
16128
16130
16131
16138
16142
16143
16146
16147
16149
16154
16156
16159
16160
16161
16163
16167
16172
16173
16174
16177
16182
16183
16184
16185
16188
16192
16196
16197
16199
16201
16202
16206
16207
16211
16214
16216
16217
16218
16219
16220
16224
16228
16232
16234
16236
16238
16244
16247
16248
16250
16252
16255
16256
1625

21254
21255
21256
21257
21258
21259
21261
21263
21264
21265
21266
21268
21269
21270
21273
21275
21277
21278
21279
21280
21283
21284
21286
21287
21289
21290
21291
21292
21293
21294
21295
21296
21297
21299
21300
21303
21305
21310
21313
21314
21317
21322
21323
21328
21332
21337
21341
21346
21347
21348
21354
21356
21358
21359
21360
21361
21362
21370
21375
21376
21380
21384
21390
21393
21394
21396
21398
21402
21408
21409
21411
21418
21419
21422
21423
21424
21425
21427
21437
21438
21439
21440
21444
21445
21446
21447
21449
21450
21461
21463
21466
21473
21476
21478
21485
21486
21487
21488
21489
21491
21493
21495
21497
21501
21503
21509
21515
21517
21524
21530
21535
21536
21537
21540
21547
21549
21550
21551
21552
21558
21560
21561
21562
21565
21566
21567
21570
21572
21574
21575
21576
21577
21578
21580
21581
21582
21583
21585
21586
21589
21591
21592
21593
21594
21595
21596
21597
21598
21600
21601
21602
21603
21606
21607
21608
21609
21610
21611
21612
21613
21614
21615
21617
21618
21620
21622
2162

26835
26836
26838
26839
26840
26842
26844
26847
26849
26850
26851
26854
26855
26856
26860
26862
26864
26867
26870
26877
26881
26884
26888
26891
26893
26896
26900
26901
26902
26903
26905
26906
26907
26908
26909
26911
26912
26914
26916
26918
26919
26921
26923
26929
26932
26933
26936
26937
26939
26940
26941
26942
26944
26947
26948
26949
26957
26958
26962
26964
26965
26967
26969
26973
26975
26978
26982
26987
26989
26996
26998
26999
27001
27003
27004
27005
27006
27007
27008
27009
27010
27011
27012
27013
27015
27016
27017
27018
27020
27022
27026
27031
27033
27035
27039
27040
27042
27043
27044
27045
27047
27048
27050
27051
27053
27055
27056
27058
27059
27060
27067
27071
27074
27081
27086
27088
27090
27093
27094
27095
27096
27099
27103
27104
27105
27106
27108
27109
27110
27111
27112
27114
27115
27116
27117
27119
27123
27124
27125
27126
27128
27129
27131
27133
27134
27135
27136
27138
27139
27140
27141
27142
27144
27148
27149
27150
27153
27154
27155
27156
27157
27158
27159
27160
27161
27162
2716

32005
32015
32016
32017
32018
32023
32024
32025
32029
32035
32036
32039
32042
32046
32047
32049
32052
32053
32056
32057
32058
32059
32061
32062
32063
32065
32066
32067
32070
32073
32074
32075
32076
32077
32078
32079
32080
32081
32082
32083
32084
32085
32086
32087
32088
32089
32090
32091
32094
32099
32100
32101
32106
32111
32112
32120
32125
32128
32129
32130
32134
32135
32136
32143
32144
32150
32151
32158
32159
32163
32164
32165
32167
32168
32169
32171
32172
32173
32174
32175
32177
32178
32179
32180
32182
32183
32184
32186
32188
32190
32191
32192
32194
32195
32196
32198
32200
32201
32203
32204
32205
32206
32207
32210
32211
32212
32213
32215
32216
32217
32218
32219
32220
32222
32223
32224
32225
32226
32229
32231
32232
32235
32236
32238
32240
32242
32243
32246
32250
32254
32255
32258
32259
32261
32266
32269
32271
32272
32273
32274
32278
32280
32282
32283
32284
32285
32286
32287
32294
32295
32296
32297
32300
32301
32303
32304
32306
32307
32309
32310
32312
32322
32324
32325
32327
32328
3232

37094
37095
37096
37097
37099
37100
37101
37104
37105
37106
37108
37109
37111
37112
37113
37117
37118
37124
37134
37136
37144
37145
37147
37148
37152
37155
37156
37165
37166
37167
37170
37175
37176
37177
37183
37184
37185
37190
37191
37192
37202
37214
37220
37222
37223
37226
37228
37229
37230
37232
37233
37234
37235
37236
37238
37240
37241
37243
37251
37254
37260
37261
37263
37269
37270
37273
37276
37277
37278
37279
37282
37283
37284
37292
37293
37296
37297
37298
37300
37301
37305
37307
37309
37310
37314
37319
37320
37321
37324
37330
37331
37332
37333
37335
37336
37337
37338
37339
37343
37344
37346
37347
37348
37349
37350
37351
37352
37355
37356
37357
37358
37359
37360
37362
37363
37364
37365
37366
37369
37372
37374
37375
37376
37378
37381
37384
37390
37391
37393
37394
37397
37399
37402
37403
37404
37405
37406
37411
37412
37413
37414
37416
37417
37418
37421
37424
37425
37429
37431
37434
37437
37438
37441
37445
37446
37448
37449
37450
37451
37454
37455
37457
37461
37463
37465
37468
3746

IndexError: single positional indexer is out-of-bounds

In [111]:
len(cor) == len(pval)

True

In [112]:
# 데이터프레임화
res = pd.DataFrame({'correlation': cor, 'p_value':pval})

In [113]:
# 상관계수 평균
np.nanmean(cor)

0.15346882651926264

In [114]:
# p-value 평균
np.nanmean(pval)

0.8339761497632444